# Подготовка данных

In [1]:
import sqlite3

import pandas as pd
import numpy as np
import ta

Загрузка данных из БД созданной заранее (pj2)

In [2]:
# Установить соединение с базой данных
conn = sqlite3.connect(r'C:\Users\Alkor\gd\data_quote_db\BR_futures_minute.db')

# SQL-запрос для извлечения данных
# query = "SELECT TRADEDATE, VOLUME FROM Minute"
query = "SELECT * FROM Minute"

# Загрузить данные в DataFrame
df = pd.read_sql_query(query, conn)

# Закрыть соединение
conn.close()

# Прописные в названиях колонок
df.columns = map(str.lower, df.columns)

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,BRF5_2015,57.05,55.12,57.05,55.31,381,2015-01-16
1,2015-01-05 10:01:00,BRF5_2015,55.37,55.32,55.40,55.38,202,2015-01-16
2,2015-01-05 10:02:00,BRF5_2015,55.40,55.32,55.44,55.32,105,2015-01-16
3,2015-01-05 10:03:00,BRF5_2015,55.32,55.32,55.39,55.37,131,2015-01-16
4,2015-01-05 10:04:00,BRF5_2015,55.38,55.38,55.43,55.43,51,2015-01-16
...,...,...,...,...,...,...,...,...
2033567,2025-01-17 23:45:00,BRG5,80.71,80.71,80.72,80.72,52,2025-02-03
2033568,2025-01-17 23:46:00,BRG5,80.72,80.71,80.72,80.71,44,2025-02-03
2033569,2025-01-17 23:47:00,BRG5,80.71,80.66,80.72,80.71,239,2025-02-03
2033570,2025-01-17 23:48:00,BRG5,80.71,80.66,80.75,80.69,290,2025-02-03


При просмотре данных заметил, что есть мусорные данные в начале торгового дня где open, high, low, close равны друг другу. Это 59 минута премаркета.
Проверка таких данных.

In [3]:
# Преобразуем 'tradedate' в datetime формат, если это ещё не сделано
df['tradedate'] = pd.to_datetime(df['tradedate'])

# Фильтрация по условиям
filtered_df = df[
    # # Все значения равны друг другу
    # (df['open'] == df['high']) & 
    # (df['open'] == df['low']) &
    # (df['open'] == df['close']) &
    # Минуты равны 59
    (df['tradedate'].dt.minute == 59) &  
    # Первая строка за дату
    (df.groupby(df['tradedate'].dt.date)['tradedate'].rank(method='first') == 1)  
]

filtered_df

,tradedate,secid,open,low,high,close,volume,lsttrade
1711356,2023-07-10 08:59:00,BRQ3,78.33,78.33,78.39,78.33,319,2023-08-01
1712222,2023-07-11 08:59:00,BRQ3,78.22,78.22,78.22,78.22,17,2023-08-01
1713091,2023-07-12 08:59:00,BRQ3,79.44,79.44,79.53,79.53,26,2023-08-01
1713957,2023-07-13 08:59:00,BRQ3,80.49,80.48,80.49,80.49,22,2023-08-01
1714828,2023-07-14 08:59:00,BRQ3,81.59,81.59,81.59,81.59,113,2023-08-01
...,...,...,...,...,...,...,...,...
2029517,2025-01-13 09:59:00,BRG5,81.28,81.11,81.28,81.28,1738,2025-02-03
2030328,2025-01-14 09:59:00,BRG5,80.47,80.47,80.59,80.49,295,2025-02-03
2031139,2025-01-15 09:59:00,BRG5,80.26,80.24,80.26,80.26,265,2025-02-03
2031950,2025-01-16 09:59:00,BRG5,82.17,82.17,82.17,82.17,30,2025-02-03


Это премаркет который потом помешает при конвертации в 5 минутные бара. Удаляем эти строки.

In [4]:
df = (
    df.merge(filtered_df, how='outer', indicator=True)
    .query('_merge == "left_only"').drop(columns=['_merge'])
    )

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,BRF5_2015,57.05,55.12,57.05,55.31,381,2015-01-16
1,2015-01-05 10:01:00,BRF5_2015,55.37,55.32,55.40,55.38,202,2015-01-16
2,2015-01-05 10:02:00,BRF5_2015,55.40,55.32,55.44,55.32,105,2015-01-16
3,2015-01-05 10:03:00,BRF5_2015,55.32,55.32,55.39,55.37,131,2015-01-16
4,2015-01-05 10:04:00,BRF5_2015,55.38,55.38,55.43,55.43,51,2015-01-16
...,...,...,...,...,...,...,...,...
2033567,2025-01-17 23:45:00,BRG5,80.71,80.71,80.72,80.72,52,2025-02-03
2033568,2025-01-17 23:46:00,BRG5,80.72,80.71,80.72,80.71,44,2025-02-03
2033569,2025-01-17 23:47:00,BRG5,80.71,80.66,80.72,80.71,239,2025-02-03
2033570,2025-01-17 23:48:00,BRG5,80.71,80.66,80.75,80.69,290,2025-02-03


# Конвертирование 1 минутных котировок в 5 минутные

In [5]:
# Убедимся, что колонка  tradedate имеет тип datetime
df["tradedate"] = pd.to_datetime(df["tradedate"])

# Устанавливаем  tradedate как индекс
df.set_index("tradedate", inplace=True)

# Агрегирование до 5-минутных баров
df_m5 = df.resample("5min").agg({
    "open": "first",    # Первая цена в периоде
    "high": "max",      # Максимальная цена
    "low": "min",       # Минимальная цена
    "close": "last",    # Последняя цена
    "volume": "sum"    # Суммарный объём
})

# Сбрасываем индекс
df_m5 = df_m5.reset_index()

# Стереть строки с NaN
df_m5.dropna(inplace=True)

# Переиндексация
df_m5 = df_m5.reset_index(drop=True)

df_m5

,tradedate,open,high,low,close,volume
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,870
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,97
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,392
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,671
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,159
...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,400
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,666
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,626
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,645


### TVI

In [6]:
def calculate_ema(series, period):
    """
    Вычисляет экспоненциальное скользящее среднее (EMA) 
    для указанного ряда данных.
    """
    return series.ewm(span=period, adjust=False).mean()

def add_tvi_column(df, r=12, s=12, u=5, point=0.0001):
    """
    Добавляет колонку TVI в DataFrame.
    
    :param df: DataFrame с колонками ['open', 'close', 'volume']
    :param r: Период для EMA для UpTicks и DownTicks
    :param s: Период для второго EMA
    :param u: Период для EMA на TVI_calculate
    :param point: Минимальное изменение цены (параметр MyPoint в MQL4)
    :return: DataFrame с добавленной колонкой 'TVI'
    """
    # Проверяем, что необходимые колонки существуют
    required_columns = ['open', 'close', 'volume']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"DataFrame должен содержать колонку '{col}'")

    # Вычисляем UpTicks и DownTicks
    df['UpTicks'] = (df['volume'] + (df['close'] - df['open']) / point) / 2
    df['DownTicks'] = df['volume'] - df['UpTicks']

    # EMA для UpTicks и DownTicks
    df['EMA_UpTicks'] = calculate_ema(df['UpTicks'], r)
    df['EMA_DownTicks'] = calculate_ema(df['DownTicks'], r)

    # Второе EMA для UpTicks и DownTicks
    df['DEMA_UpTicks'] = calculate_ema(df['EMA_UpTicks'], s)
    df['DEMA_DownTicks'] = calculate_ema(df['EMA_DownTicks'], s)

    # Расчет TVI_calculate
    df['TVI_calculate'] = 100.0 * (df['DEMA_UpTicks'] - df['DEMA_DownTicks']) / (
        df['DEMA_UpTicks'] + df['DEMA_DownTicks']
    )

    # EMA на TVI_calculate
    df['TVI'] = calculate_ema(df['TVI_calculate'], u)

    # Удаляем промежуточные колонки, если они не нужны
    df.drop(
        columns=['UpTicks', 'DownTicks', 'EMA_UpTicks', 'EMA_DownTicks', 
        'DEMA_UpTicks', 'DEMA_DownTicks', 'TVI_calculate'], inplace=True
        )

    return df

# Добавляем колонку TVI
df_m5 = add_tvi_column(df_m5, r=12, s=12, u=5, point=0.1)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,870,-1.862069
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,97,-1.860211
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,392,-1.851780
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,671,-1.829566
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,159,-1.798802
...,...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,400,-0.004999
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,666,-0.003343
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,626,-0.001872
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,645,-0.000517


## GannHiLo

In [7]:
def calculate_sma(series, period):
    """
    Вычисляет простую скользящую среднюю (SMA).
    """
    return series.rolling(window=period, min_periods=1).mean()

def add_gann_hilo_column_optimized(df, period=10):
    """
    Оптимизированная версия добавления индикатора GannHiLo в DataFrame.
    
    :param df: DataFrame с колонками ['close', 'high', 'low']
    :param period: Период для расчета SMA
    :return: DataFrame с добавленными колонками 'Gann_HiLo'
    """
    # Проверка на необходимые колонки
    required_columns = ['close', 'high', 'low']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(
            f"DataFrame должен содержать колонки: {required_columns}"
            )

    # Расчет SMA для high и low
    sma_high = calculate_sma(df['high'], period)
    sma_low = calculate_sma(df['low'], period)

    # Вычисление GannHiLo векторным способом
    gann_hilo = (
        (df['close'] > sma_high.shift(1)).astype(int) - 
        (df['close'] < sma_low.shift(1)).astype(int)
        )

    # Добавляем колонку Gann_HiLo в DataFrame
    df['Gann_HiLo'] = gann_hilo

    return df

# Добавляем GannHiLo индикатор
df_m5 = add_gann_hilo_column_optimized(df_m5, period=10)

# Результат
df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,870,-1.862069,0
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,97,-1.860211,0
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,392,-1.851780,0
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,671,-1.829566,-1
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,159,-1.798802,0
...,...,...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,400,-0.004999,1
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,666,-0.003343,1
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,626,-0.001872,1
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,645,-0.000517,1


## T3 (T3ma) (8, 0.618)

In [8]:
def calculate_t3ma(df, period=8, b=0.618):
    """
    Рассчитывает индикатор T3MA и добавляет его как новую колонку в DataFrame.

    :param df: DataFrame с колонкой 'close'
    :param period: Период T3MA
    :param b: Параметр T3MA (значение сглаживания)
    :return: DataFrame с добавленной колонкой 'T3MA'
    """
    # Проверка на наличие колонки 'close'
    if 'close' not in df.columns:
        raise ValueError("DataFrame должен содержать колонку 'close'")

    close = df['close'].values

    # Коэффициенты
    b2 = b ** 2
    b3 = b2 * b
    c1 = -b3
    c2 = 3 * (b2 + b3)
    c3 = -3 * (2 * b2 + b + b3)
    c4 = 1 + 3 * b + b3 + 3 * b2
    n = max(1, 1 + 0.5 * (period - 1))
    w1 = 2 / (n + 1)
    w2 = 1 - w1

    # Создание временных массивов
    e1 = np.zeros_like(close)
    e2 = np.zeros_like(close)
    e3 = np.zeros_like(close)
    e4 = np.zeros_like(close)
    e5 = np.zeros_like(close)
    e6 = np.zeros_like(close)

    # Векторизованный расчет экспоненциальных сглаживаний
    e1[0] = close[0]
    for i in range(1, len(close)):
        e1[i] = w1 * close[i] + w2 * e1[i - 1]

    e2[0] = e1[0]
    for i in range(1, len(e1)):
        e2[i] = w1 * e1[i] + w2 * e2[i - 1]

    e3[0] = e2[0]
    for i in range(1, len(e2)):
        e3[i] = w1 * e2[i] + w2 * e3[i - 1]

    e4[0] = e3[0]
    for i in range(1, len(e3)):
        e4[i] = w1 * e3[i] + w2 * e4[i - 1]

    e5[0] = e4[0]
    for i in range(1, len(e4)):
        e5[i] = w1 * e4[i] + w2 * e5[i - 1]

    e6[0] = e5[0]
    for i in range(1, len(e5)):
        e6[i] = w1 * e5[i] + w2 * e6[i - 1]

    # Итоговый расчет T3
    t3 = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3

    # Добавление результата в DataFrame
    df['T3MA'] = t3
    return df

# Рассчитываем T3MA и добавляем в DataFrame
df_m5 = calculate_t3ma(df_m5, period=8, b=0.618)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,870,-1.862069,0,55.430000
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,97,-1.860211,0,55.432601
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,392,-1.851780,0,55.421160
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,671,-1.829566,-1,55.390131
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,159,-1.798802,0,55.358814
...,...,...,...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,400,-0.004999,1,80.660887
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,666,-0.003343,1,80.677402
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,626,-0.001872,1,80.688638
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,645,-0.000517,1,80.698574


# CCI (20)

In [9]:
# Вычисление CCI с периодом 20
df_m5['CCI_20'] = ta.trend.cci(
    high=df_m5['high'], low=df_m5['low'], close=df_m5['close'], window=20
    )

df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA,CCI_20
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,870,-1.862069,0,55.430000,NaN
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,97,-1.860211,0,55.432601,NaN
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,392,-1.851780,0,55.421160,NaN
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,671,-1.829566,-1,55.390131,NaN
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,159,-1.798802,0,55.358814,NaN
...,...,...,...,...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,400,-0.004999,1,80.660887,-7.358352
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,666,-0.003343,1,80.677402,-13.493569
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,626,-0.001872,1,80.688638,-12.662446
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,645,-0.000517,1,80.698574,0.360750


# Сигналы индикаторов

In [10]:
# Добавляем новую колонку с сигналом TVI
df_m5['TVI_S'] = df_m5['TVI'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом T3MA
df_m5['T3_S'] = df_m5['T3MA'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом от CCI
df_m5['CCI_S'] = df_m5['CCI_20'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

df_m5 = df_m5[['tradedate', 'open', 'high', 'low', 'close', 
               'TVI_S', 'CCI_S', 'T3_S', 'Gann_HiLo']]

df_m5 = df_m5.copy()

# Переименование колонок
df_m5.rename(
    columns={'TVI_S': 'tvi', 'CCI_S': 'cci', 'T3_S': 't3', 'Gann_HiLo': 'ghl'}, 
    inplace=True
    )

df_m5

,tradedate,open,high,low,close,tvi,cci,t3,ghl
0,2015-01-05 10:00:00,57.05,57.05,55.12,55.43,0,0,0,0
1,2015-01-05 10:05:00,55.43,55.46,55.37,55.45,1,0,1,0
2,2015-01-05 10:10:00,55.45,55.45,55.33,55.33,1,0,-1,0
3,2015-01-05 10:15:00,55.34,55.38,55.20,55.25,1,0,-1,-1
4,2015-01-05 10:20:00,55.27,55.36,55.27,55.31,1,0,-1,0
...,...,...,...,...,...,...,...,...,...
410633,2025-01-17 23:25:00,80.69,80.72,80.68,80.72,1,-1,1,1
410634,2025-01-17 23:30:00,80.71,80.72,80.67,80.68,1,-1,1,1
410635,2025-01-17 23:35:00,80.69,80.71,80.65,80.70,1,-1,1,1
410636,2025-01-17 23:40:00,80.71,80.72,80.66,80.72,1,1,1,1


### Сохранение в файл

In [11]:
df_m5.to_csv("br_m5_TVI_CCI_T3_GHL.csv", index=False)